In [ ]:
import os
from ultralytics import NAS
os.chdir("..")
ROOT = os.getcwd()
print(ROOT)


In [ ]:
from models.nas import *


In [ ]:
### class inits and other inputs

path_model = 'yolo_nas_l' 
dir_train = "/Users/mautushid/github/Cowsformer/data/cow200/yolov5/train"
dir_val = "/Users/mautushid/github/Cowsformer/data/cow200/yolov5/val"
dir_test = "/Users/mautushid/github/Cowsformer/data/cow200/yolov5/test_old"
name_task = "cow200"

data_yaml_path = "/Users/mautushid/github/Cowsformer/data/cow200/yolov5/data.yaml"
finetuned_model_path = "/Users/mautushid/github/Cowsformer/__notebooks/lms_checkpoints/yolo_l_100/n100_yolo_na_i2_exp_yolo_nas_l_100_2/ckpt_best.pth"

### Creating instance of Niche_YOLO_NAS class
my_nas = Niche_YOLO_NAS(path_model, dir_train, dir_val, dir_test, name_task)

In [ ]:
model = my_nas.load(path_model,finetuned_model_path)

In [ ]:
import supervision as sv

ds = sv.DetectionDataset.from_yolo(
    images_directory_path="/Users/mautushid/github/Cowsformer/data/cow200/yolov5/test_old/images",
    annotations_directory_path="/Users/mautushid/github/Cowsformer/data/cow200/yolov5/test_old/labels",
    data_yaml_path="/Users/mautushid/github/Cowsformer/data/cow200/yolov5/data.yaml",
    force_masks=False
)
CONFIDENCE_TRESHOLD = 0.5

predictions = {}

for image_name, image in ds.images.items():
    result = list(model.predict(
        image, conf=CONFIDENCE_TRESHOLD))[0]
    detections = sv.Detections(
        xyxy=result.prediction.bboxes_xyxy,
        #xyxy=result.prediction.bboxes_xywh,
        confidence=result.prediction.confidence,
        class_id=result.prediction.labels.astype(int)
    )
    predictions[image_name] = detections

In [ ]:
import os
'''
# Assuming `predictions` is your dictionary of results as shown above
# And assuming `output_directory` is the path to the directory where you want to save the .txt files
output_directory = 'path/to/output/directory'

# Ensure the output directory exists
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Iterate over the predictions and write them to files
for image_name, detections in predictions.items():
    # Prepare the output file path
    base_filename = os.path.splitext(image_name)[0]
    output_file_path = os.path.join(output_directory, base_filename + '.txt')

    # Open the file and write the detections
    with open(output_file_path, 'w') as file:
        for bbox, conf, class_id in zip(detections.xyxy, detections.confidence, detections.class_id):
            # Create a string for each detection
            # Format: class_id x_min y_min x_max y_max confidence
            detection_str = f"{class_id} {bbox[0]} {bbox[1]} {bbox[2]} {bbox[3]} {conf}\n"
            # Write to file
            file.write(detection_str)

print(f"All predictions have been written to {output_directory}")
'''

In [ ]:
from API import*
api = YOLO_API("/Users/mautushid/github/Cowsformer/data/cow200/yolov5") 
lbs = api.get_gt_detections(
    "test_old")
pre = api.get_pred_detections(
    "test_old/output")

In [ ]:
from evaluate import from_sv

In [ ]:
op = from_sv(pre, lbs)

In [ ]:
import pandas as pd


def make_table_row(EvalOutput, dataset_name, model_name, data_size):
    # Combine EvalOutput with the additional information
    combined_dict = {
        **EvalOutput,  # Unpack the EvalOutput dictionary
        'config': dataset_name,  # Add dataset_name as 'config'
        'model': model_name,  # Add model_name as 'model'
        'n': data_size  # Add data_size as 'n'
    }

    # Create a DataFrame from the combined dictionary
    # The index=[0] makes the dictionary values to be considered as a single row
    df = pd.DataFrame([combined_dict])

    return df



dataset_name = "cow200"
model_name = "yolo_nas_l"
data_size = "200"

df = make_table_row(op, dataset_name, model_name, data_size)

# Display the DataFrame
print(df)